In [2]:
import pandas as pd

## Chargement des données 

In [24]:
df = pd.read_csv("../DEMOCRATIE_ET_CITOYENNETE.csv", low_memory = False)
df.shape
#df.head()


(28384, 46)

## Récupération des noms des colonnes pour savoir lesquelles utilisées

https://moonbooks.org/Articles/Comment-selectionner-une-ou-plusieurs-colonnes-de-donnees-dune-data-frame-avec-pandas-sous-python-/

In [32]:
for idx,column in enumerate(df.columns):
    print(idx,column)

0 reference
1 title
2 createdAt
3 publishedAt
4 updatedAt
5 trashed
6 trashedStatus
7 authorId
8 authorType
9 authorZipCode
10 En qui faites-vous le plus confiance pour vous faire représenter dans la société et pourquoi ?
11 En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?
12 Si oui, à quel type d'associations ou d'organisations ? Et avec quel rôle ?
13 Que faudrait-il faire pour renouer le lien entre les citoyens et les élus qui les représentent ?
14 Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :
15 Pourquoi ?
16 Que faudrait-il faire pour mieux représenter les différentes sensibilités politiques ?
17 Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?
18 Si oui, lesquels ?
19 Que pensez-vous de la participation des citoyens aux élections et comment les inciter à y participer davantage ?
20 Faut-il prendre

In [38]:
print(df.iloc[:,40:44])

                                    Si oui, lesquelles ?  \
0                                                    NaN   
1                                                    NaN   
2                                                    NaN   
3                                                    NaN   
4                                                    NaN   
...                                                  ...   
28379                                                NaN   
28380                                                NaN   
28381                                                NaN   
28382  toute allocations doit avoir une contrepartie ...   
28383                                                NaN   

      Que pensez-vous de la situation de l'immigration en France aujourd'hui et de la politique migratoire ? Quelles sont, selon vous, les critères à mettre en place pour définir la politique migratoire ?  \
0                                                    NaN               

## Suppression des colonnes inutiles

In [17]:
df_mod = df.drop(["createdAt"], axis =1)
df_mod.head()

,reference,title,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,authorZipCode,En qui faites-vous le plus confiance pour vous faire représenter dans la société et pourquoi ?,...,Quel pourrait être le rôle de chacun pour faire reculer les incivilités dans la société ?,Quelles sont les discriminations les plus répandues dont vous êtes témoin ou victime ?,Que faudrait-il faire pour lutter contre ces discriminations et construire une société plus solidaire et plus tolérante ?,Pensez-vous qu'il faille instaurer des contreparties aux différentes allocations de solidarité ?,"Si oui, lesquelles ?","Que pensez-vous de la situation de l'immigration en France aujourd'hui et de la politique migratoire ? Quelles sont, selon vous, les critères à mettre en place pour définir la politique migratoire ?","En matière d'immigration, une fois nos obligations d'asile remplies, souhaitez-vous que nous puissions nous fixer des objectifs annuels définis par le Parlement ?",Que proposez-vous afin de répondre à ce défi qui va durer ?,"Quelles sont, selon vous, les modalités d'intégration les plus efficaces et les plus justes à mettre en place aujourd'hui dans la société ?",Y a-t-il d'autres points sur la démocratie et la citoyenneté sur lesquels vous souhaiteriez vous exprimer ?
0,1-2,Les augmentations de rémunérations,2019-01-22 09:38:41,NaN,False,NaN,VXNlcjo4Mjc4NzQxYS0xZTFkLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,79190,Le citoyen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afin d’éviter de creuser les inégalités ne plu...
1,1-3,rénover l'enquête publique pour en faire un vr...,2019-01-22 09:40:04,2019-01-22 09:41:00,False,NaN,VXNlcjo4OWQ3MzE5My0xZDYwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,1800,Un instrument de démocratie locale à modernise...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1-4,Limitations de vitesse et sécurité routière,2019-01-22 09:44:04,NaN,False,NaN,VXNlcjowMzYyMTUyNy0xZDEyLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,44300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nous proposons le retour à la limitation de vi...
3,1-5,Pour un nouveau contrat citoyen,2019-01-22 09:44:05,NaN,False,NaN,VXNlcjo4YWJlYzBmOS0xZGE3LTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,29490,Voir l'intégralité de la proposition dans la d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POUR UN NOUVEAU CONTRAT CITOYEN ...
4,1-6,Développer les démarches de budget participati...,2019-01-22 09:45:43,NaN,False,NaN,VXNlcjo2Nzc5MjE4OC0xZTIxLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,94300,"député, maire, moi même",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
